<a href="https://colab.research.google.com/github/sathwikkumar1207/Smart-Emotion-Detection/blob/main/smart_emotion_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy deepface fastapi opencv-python uvicorn pillow pyngrok
!pip install python-multipart
!ngrok authtoken 2tigyiCSpJnAnV82VnQniUearKM_45G7teJBzbbh2dWjpbxVX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=7b4275fb7c53181b13d051b1fa5040509ca1aa782995f78e1cd5813ceb591bca
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
Authtoken saved to configuration file: /root/.config/ng

In [ ]:
import numpy as np
import cv2
from fastapi import FastAPI, File, UploadFile
from deepface import DeepFace
import uvicorn
from pyngrok import ngrok
from pathlib import Path
from PIL import Image
import io
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

# Initialize FastAPI app
app = FastAPI()

# Enable CORS to allow frontend requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins, replace with frontend URL if needed
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configure upload folder
UPLOAD_FOLDER = Path("uploads")
UPLOAD_FOLDER.mkdir(exist_ok=True)

# Emotion Context Dictionary
EMOTION_CONTEXT = {
    "angry": "The person appears to be angry. This could be due to frustration, conflict, or dissatisfaction.",
    "disgust": "The expression suggests disgust. This might be triggered by an unpleasant sight, smell, or thought.",
    "fear": "Fear is detected, which could be caused by a threat, danger, or overwhelming anxiety.",
    "happy": "A happy expression! The person might be experiencing joy, excitement, or a sense of achievement.",
    "sad": "The individual seems sad. Possible reasons could be personal loss, disappointment, or loneliness.",
    "surprise": "The person looks surprised, which could be due to an unexpected event, shock, or sudden realization.",
    "neutral": "A neutral expression. This could indicate calmness, focus, or a lack of strong emotions.",
}

# Emotion Colors
EMOTION_COLORS = {
    "angry": "#FF0000",
    "disgust": "#008000",
    "fear": "#800080",
    "happy": "#FFD700",
    "sad": "#0000FF",
    "surprise": "#FFA500",
    "neutral": "#808080",
}

# Emotion Emojis
EMOTION_EMOJIS = {
    "angry": "😡",
    "disgust": "🤢",
    "fear": "😨",
    "happy": "😃",
    "sad": "😢",
    "surprise": "😲",
    "neutral": "😐",
  }

# Function to determine intensity level
def get_intensity_level(percentage):
    if percentage < 50:
        return "Low 🟢"
    elif 50 <= percentage < 70:
        return "Normal 🟡"
    elif 70 <= percentage < 85:
        return "High 🟠"
    else:
        return "Extreme 🔴"

# Function to convert image file to NumPy array
def read_image_as_numpy(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = image.convert("RGB")  # Convert to RGB
    img_np = np.array(image)  # Convert to NumPy array
    return img_np

# API endpoint for image upload and emotion detection
@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    try:
        # Read image as NumPy array
        image_data = await file.read()
        img_np = read_image_as_numpy(image_data)

        # Save the image temporarily
        filepath = UPLOAD_FOLDER / file.filename
        cv2.imwrite(str(filepath), cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))

        # Perform emotion recognition
        analysis = DeepFace.analyze(img_path=str(filepath), actions=["emotion"])

        # Extract dominant emotion and percentage
        emotions = analysis[0]["emotion"]

        # Convert all NumPy float32 to Python float
        emotions = {k: float(v) for k, v in emotions.items()}

        dominant_emotion = max(emotions, key=emotions.get)
        confidence = round(emotions[dominant_emotion], 2)

        # Get intensity level
        intensity = get_intensity_level(confidence)

        # Get context explanation
        emotion_context = EMOTION_CONTEXT.get(dominant_emotion, "Emotion not recognized.")

        # Create response
        response = {
            "emotion": dominant_emotion,
            "percentage": confidence,
            "emoji": EMOTION_EMOJIS.get(dominant_emotion, ""),
            "color": EMOTION_COLORS.get(dominant_emotion, "#000000"),
            "intensity": intensity,
            "context": emotion_context
        }

        return JSONResponse(content=response)

    except Exception as e:
        return JSONResponse(content={"error": str(e)})

# Start the ngrok tunnel and run FastAPI server
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)
# 

25-03-24 08:09:58 - Directory /root/.deepface has been created
25-03-24 08:09:58 - Directory /root/.deepface/weights has been created
Public URL: https://18ea-35-185-55-176.ngrok-free.app


INFO:     Started server process [500]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


25-03-24 08:10:40 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 70.2MB/s]


INFO:     27.116.21.222:0 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     2401:4900:62d6:715b:60b3:8cb6:78ec:39fb:0 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     2401:4900:62d6:715b:60b3:8cb6:78ec:39fb:0 - "POST /upload/ HTTP/1.1" 200 OK
